In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3,4,5,6"

In [3]:
import vllm, torch, os, subprocess, importlib
print("vLLM:", vllm.__version__, vllm.__file__)
print("Torch:", torch.__version__, torch.version.cuda)

/gpfs01/home/ppxac9/MLiS-Placement-Thesis/pipeline/optimizing_inference/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 08-11 23:53:18 [__init__.py:239] Automatically detected platform cuda.


2025-08-11 23:53:19,403	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


vLLM: 0.8.4 /gpfs01/home/ppxac9/MLiS-Placement-Thesis/pipeline/optimizing_inference/.venv/lib/python3.11/site-packages/vllm/__init__.py
Torch: 2.6.0+cu124 12.4


In [4]:
from vllm import LLM, SamplingParams

llm = LLM(
        model="Qwen/Qwen3-30B-A3B-Instruct-2507",
        tensor_parallel_size=4
    )


INFO 08-11 23:54:04 [config.py:689] This model supports multiple tasks: {'generate', 'reward', 'embed', 'classify', 'score'}. Defaulting to 'generate'.
INFO 08-11 23:54:04 [config.py:1713] Defaulting to use mp for distributed inference
INFO 08-11 23:54:04 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 08-11 23:54:05 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='Qwen/Qwen3-30B-A3B-Instruct-2507', speculative_config=None, tokenizer='Qwen/Qwen3-30B-A3B-Instruct-2507', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=262144, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning

Loading safetensors checkpoint shards:   0% Completed | 0/16 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   6% Completed | 1/16 [00:00<00:11,  1.26it/s]
Loading safetensors checkpoint shards:  12% Completed | 2/16 [00:01<00:10,  1.30it/s]
Loading safetensors checkpoint shards:  19% Completed | 3/16 [00:02<00:09,  1.37it/s]
Loading safetensors checkpoint shards:  25% Completed | 4/16 [00:02<00:08,  1.46it/s]
Loading safetensors checkpoint shards:  31% Completed | 5/16 [00:03<00:07,  1.52it/s]
Loading safetensors checkpoint shards:  38% Completed | 6/16 [00:05<00:09,  1.03it/s]
Loading safetensors checkpoint shards:  44% Completed | 7/16 [00:05<00:07,  1.13it/s]
Loading safetensors checkpoint shards:  50% Completed | 8/16 [00:06<00:07,  1.14it/s]
Loading safetensors checkpoint shards:  56% Completed | 9/16 [00:07<00:05,  1.25it/s]
Loading safetensors checkpoint shards:  62% Completed | 10/16 [00:07<00:04,  1.35it/s]
Loading safetensors checkpoint shards:  69% Completed | 11/16

(VllmWorker rank=0 pid=3300986) INFO 08-11 23:54:25 [loader.py:458] Loading weights took 12.03 seconds
(VllmWorker rank=0 pid=3300986) INFO 08-11 23:54:25 [gpu_model_runner.py:1291] Model loading took 14.3001 GiB and 12.707204 seconds
(VllmWorker rank=1 pid=3301010) INFO 08-11 23:54:26 [loader.py:458] Loading weights took 12.61 seconds
(VllmWorker rank=1 pid=3301010) INFO 08-11 23:54:26 [gpu_model_runner.py:1291] Model loading took 14.3001 GiB and 13.652692 seconds
(VllmWorker rank=2 pid=3301047) INFO 08-11 23:54:26 [loader.py:458] Loading weights took 12.86 seconds
(VllmWorker rank=2 pid=3301047) INFO 08-11 23:54:27 [gpu_model_runner.py:1291] Model loading took 14.3001 GiB and 14.169387 seconds
(VllmWorker rank=3 pid=3301077) INFO 08-11 23:54:27 [loader.py:458] Loading weights took 13.73 seconds
(VllmWorker rank=3 pid=3301077) INFO 08-11 23:54:28 [gpu_model_runner.py:1291] Model loading took 14.3001 GiB and 15.316636 seconds
(VllmWorker rank=1 pid=3301010) INFO 08-11 23:54:37 [backend

In [5]:

prompts = [
    "What is the capital of France?",
    "What is the coup capital?"
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=100)


In [6]:
outputs = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 2/2 [00:18<00:00,  9.31s/it, est. speed input: 0.70 toks/s, output: 10.74 toks/s]


In [8]:
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Prompt: 'What is the capital of France?', Generated text: ' Also, can you explain the significance of this city in French history and culture?\n\nThe capital of France is Paris. Paris has been the political, cultural, and economic center of France for centuries. It played a crucial role in the French Revolution, which began in 1789 and led to the overthrow of the monarchy. Paris is also home to iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. The city has been a hub for art'
Prompt: 'What is the coup capital?', Generated text: '  Is it a verb?\n\nThe phrase **"coup capital"** is not a standard or commonly used term in English, and it is **not a verb**. Let\'s break it down:\n\n### 1. **"Coup"**:\n- As a **noun**, "coup" refers to a sudden, decisive, and often violent or illegal seizure of power, especially in a government (e.g., *a military coup*).\n- As a **verb**, "to'


In [9]:
from pydantic import BaseModel, Field, RootModel
from typing import Union, Literal
from enum import Enum

# Separate enums for clarity and type safety
class MathOperator(str, Enum):
    ADD = "ADD"
    MULTIPLY = "MULTIPLY"
    DIVIDE = "DIVIDE"
    CEIL = "CEIL"
    
class Comparator(str, Enum):
    GREATER_THAN = "GREATER_THAN"
    LESS_THAN = "LESS_THAN"
    EQUAL_TO = "EQUAL_TO"

class Units(str, Enum):
    HOURS = "HOURS"
    MINUTES = "MINUTES"
    EUROS = "EUROS"
    PERCENT = "PERCENT"
    UNITLESS = "UNITLESS"
    
# --- Node Definitions ---

class ValueNode(BaseModel):
    type: Literal["VALUE"] = "VALUE"
    value: float
    description: str
    unit: Units

class VariableNode(BaseModel):
    type: Literal["VARIABLE"] = "VARIABLE"
    name: str 
    description: str
    unit: Units

class BinaryOpNode(BaseModel):
    """Node for mathematical operations that produce a number."""
    type: Literal["BINARY_OPERATION"] = "BINARY_OPERATION"
    operator: MathOperator
    left: 'AnyNode'
    right: 'AnyNode'

class ComparisonNode(BaseModel):
    """Node for comparison operations that produce a boolean."""
    type: Literal["COMPARISON"] = "COMPARISON"
    operator: Comparator
    left: 'AnyNode'
    right: 'AnyNode'

class ConditionalNode(BaseModel):
    """Node for if-then-else logic."""
    type: Literal["CONDITIONAL"] = "CONDITIONAL"
    condition: ComparisonNode # Condition must be a comparison
    if_true: 'AnyNode'
    if_false: 'AnyNode'

# --- Recursive Setup ---

AnyNode = Union[
    ValueNode, 
    VariableNode, 
    BinaryOpNode, 
    ConditionalNode
]

# Use model_rebuild() to safely resolve all forward references
BinaryOpNode.model_rebuild()
ConditionalNode.model_rebuild()
ComparisonNode.model_rebuild()

class Node(RootModel):
    root: BinaryOpNode

In [11]:
json_schema = Node.model_json_schema()

In [13]:

from vllm.sampling_params import GuidedDecodingParams
guided_decoding_params = GuidedDecodingParams(json=json_schema)

In [14]:
sampling_params = SamplingParams(guided_decoding=guided_decoding_params)


In [15]:
from pydantic import BaseModel, Field, field_validator
from typing import List, Dict, Any, Optional
from enum import Enum


class ParameterStatus(str, Enum):
    """An enumeration for clear, explicit parameter statuses."""
    KNOWN = "KNOWN"
    SYMBOLIC = "SYMBOLIC"

class ParameterDetail(BaseModel):
    """A structured model to describe each parameter identified from the query."""
    name: str = Field(..., description="The name of the parameter.")
    status: ParameterStatus = Field(..., description="Whether the parameter's value is known from the query or is a symbolic variable.")
    value: Optional[Any] = Field(None, description="The actual value of the parameter, if its status is 'KNOWN'. Must be null if status is 'SYMBOLIC'.")


class ReasoningSchemaStep1(BaseModel):
    query_parameters: List[ParameterDetail] = Field(
        ...,
        description="A structured list of all parameters identified from the query and their status."
    )
class ReasoningSchemaStep2(BaseModel):
    """
    A simplified schema for Step 2 that captures all constants and rules as a simple list of descriptive strings.
    """
    identified_constants_and_rules: List[str] = Field(
        ...,
        description="A comprehensive list of all facts, constants, and conditional rules extracted from the document that are necessary for the final calculation. Each string in the list should be a self-contained, clear statement. For example: 'The rate for a Narrow Satellite stand is €32.90 per 15 minutes' or 'A 100% surcharge is applied if parking duration is between 48 and 72 hours'."
    )
class ReasoningSchemaStep3(BaseModel):
    synthesis_plan: str = Field(
        ...,
        description="A concise, step-by-step plan describing how the variables and constants are combined into the final computation graph."
    )
class ReasoningSchemaStep4(BaseModel):
    rethink: str = Field(
        ...,
        description="Final check to ensure the plan correctly uses variables and constants and handles all logic from the document."
    )


In [ ]:
outputs = llm.generate(
    prompts="Classify this sentiment: vLLM is wonderful!",
    sampling_params=sampling_params,
)

In [ ]:
import os
from pydantic import BaseModel
import guidance
litellm_desc = {
    "model_name": "Qwen/Qwen3-1.7B",
    "litellm_params": {  # params for litellm completion/embedding call
        "model": "hosted_vllm/Qwen/Qwen3-1.7B",
        "api_key": os.environ.get("VLLM_API_KEY", "NO_KEY"), # set your vLLM API key if needed
        "api_base": "http://localhost:8000/v1", # change to your vLLM API base URL
    },
}
base_lm = guidance.models.experimental.LiteLLM(model_description=litellm_desc, echo=False)